# Architecture B

<!-- MODIFIED: Updated from Architecture A to Architecture B with Transformer encoder -->

**Architecture Specifications:**
- **Image Input**: 100×100 grayscale → 224×224 RGB (ResNet-18 compatible)
- **Image Backbone**: ResNet-18 (pretrained) → 512-D image feature
- **Text Input**: Short text metadata (tokenized with subword units, e.g., BPE)
- **Text Encoder**: Transformer encoder (2–4 layers, 4–8 heads) → 512-D text embedding
- **Fusion**: Concatenate [512-D image, 512-D text] → 1024-D
- **Dropout**: p=0.3 (randomly drops ~30% of fused features during training)
- **Head**: Linear (1024 → 7), Softmax for probabilities
- **Loss**: Cross-Entropy

## Specific Imports
imports used for the specific model tasks

In [1]:
# Install scikit-learn if needed
import subprocess
import sys

try:
    from sklearn.model_selection import train_test_split
except ImportError:
    print("Installing scikit-learn...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
    from sklearn.model_selection import train_test_split
    print("scikit-learn installed successfully!")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import pandas as pd
import numpy as np
import random

import re
from datasets import load_dataset

import sys
import os
from tqdm.notebook import tqdm

import kaggle
import kagglehub
from kagglehub import KaggleDatasetAdapter
import time
import math
from collections import Counter

# Text processing library - minimal approach
from transformers import AutoTokenizer

# train_test_split is already imported above

# Set up device for GPU/CPU usage throughout the notebook
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check CUDA availability and GPU info
if torch.cuda.is_available():
    print(f"CUDA is available!")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("CUDA is not available, using CPU")


Using device: cuda:0
CUDA is available!
GPU count: 1
Current GPU: 0
GPU name: NVIDIA GeForce RTX 5060 Ti
GPU memory: 15.5 GB


In [2]:
# Text Processing using transformers library (minimal code approach)
# Initialize tokenizer - uses BPE tokenization automatically
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text, max_length=15):
    """Tokenize text using pre-trained tokenizer - minimal code"""
    encoded = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoded['input_ids'].squeeze(0)  # Return token IDs as tensor

def get_vocab_size():
    """Get vocabulary size from tokenizer"""
    return tokenizer.vocab_size


# Text Processing Functions

In [3]:
# Text Processing Functions - Using transformers library (minimal code)
# Note: tokenizer is initialized in cell 3 above

def tokenize_text(text, max_length=10):
    """Tokenize text using pre-trained tokenizer - minimal code"""
    encoded = tokenizer(
        text,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoded['input_ids'].squeeze(0)  # Return token IDs as tensor
    

# Dataset Setup

In [4]:
#Extremely simple data set class.
class OurDataSetA(Dataset):
    def __init__(self, data_directory, transform=None):
        self.data = ImageFolder(data_directory, transform=transform)

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, at_index):
        return self.data[at_index]

    @property
    def classes(self):
        return self.data.classes
#END CLASS

path = kagglehub.dataset_download("dollyprajapati182/balanced-raf-db-dataset-7575-grayscale")

print("Path to dataset files:", path)


#Strings of data directories
str_data_dir_train = path + '/train'
str_data_dir_valid = path + '/val'
str_data_dir_test  = path + '/test'


#Transform
# A- this is meant for the balanced grey-scale RAF data set
transform_a = transforms.Compose([
    transforms.ToTensor()
])

# B- this is meant for the RAF data set
transform_b = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),   
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Highly technical super nuanced and novel way of setting the transform to use.
transform_used = transform_a

#Dataset variables
dataset_train = OurDataSetA(str_data_dir_train, transform_used)
dataset_valid = OurDataSetA(str_data_dir_valid, transform_used)
dataset_test  = OurDataSetA(str_data_dir_test, transform_used)

#Label Dictionary
label_dict ={
    0:"Angry",
    1:"Disgust",
    2:"Fear",
    3:"Happy",
    4:"Neutral",
    5:"Sad",
    6:"Surprise"
}

#This just serves to take a random snippet from the dataset and display it for demo purposes
var_rand = random.randint(1,(4289*7))
image, label = dataset_train[var_rand]
print("index used: ", var_rand)
print(label_dict[label])
image

Path to dataset files: /home/amherscher/.cache/kagglehub/datasets/dollyprajapati182/balanced-raf-db-dataset-7575-grayscale/versions/1
index used:  20704
Neutral


tensor([[[0.0549, 0.1451, 0.2431,  ..., 0.9725, 0.8706, 0.9569],
         [0.3490, 0.3961, 0.4235,  ..., 0.8824, 1.0000, 0.8157],
         [0.5647, 0.5922, 0.5686,  ..., 0.9804, 0.9725, 1.0000],
         ...,
         [0.1686, 0.1255, 0.1216,  ..., 0.0039, 0.0039, 0.0039],
         [0.1843, 0.1373, 0.1333,  ..., 0.0039, 0.0039, 0.0039],
         [0.2000, 0.1529, 0.1490,  ..., 0.0039, 0.0039, 0.0039]],

        [[0.0549, 0.1451, 0.2431,  ..., 0.9725, 0.8706, 0.9569],
         [0.3490, 0.3961, 0.4235,  ..., 0.8824, 1.0000, 0.8157],
         [0.5647, 0.5922, 0.5686,  ..., 0.9804, 0.9725, 1.0000],
         ...,
         [0.1686, 0.1255, 0.1216,  ..., 0.0039, 0.0039, 0.0039],
         [0.1843, 0.1373, 0.1333,  ..., 0.0039, 0.0039, 0.0039],
         [0.2000, 0.1529, 0.1490,  ..., 0.0039, 0.0039, 0.0039]],

        [[0.0549, 0.1451, 0.2431,  ..., 0.9725, 0.8706, 0.9569],
         [0.3490, 0.3961, 0.4235,  ..., 0.8824, 1.0000, 0.8157],
         [0.5647, 0.5922, 0.5686,  ..., 0.9804, 0.9725, 1.

## Multi Modal Dataset

In [5]:
# Transformer Encoder for Text Processing (Architecture B)
# This must be defined before MultiModalEmotionClassifierB
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=3, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_encoding = self._create_positional_encoding(d_model)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=2048,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output projection to 512-D
        self.output_proj = nn.Linear(d_model, 512)
        
    def _create_positional_encoding(self, d_model, max_len=100):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)
    
    def forward(self, text_tokens):
        # Get sequence length
        seq_len = text_tokens.size(1)
        
        # Embedding + positional encoding
        embedded = self.embedding(text_tokens) * math.sqrt(self.d_model)
        embedded = embedded + self.pos_encoding[:, :seq_len, :].to(text_tokens.device)
        
        # Create attention mask for padding tokens
        attention_mask = (text_tokens != 0).float()
        
        # Transformer encoding
        transformer_output = self.transformer(embedded, src_key_padding_mask=attention_mask == 0)
        
        # Global average pooling (mean of non-padded tokens)
        mask = attention_mask.unsqueeze(-1).expand_as(transformer_output)
        masked_output = transformer_output * mask
        text_features = masked_output.sum(dim=1) / mask.sum(dim=1)
        
        # Project to 512-D
        text_features = self.output_proj(text_features)
        
        return text_features


In [6]:
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        # Use torchvision instead of timm
        self.base_model = torchvision.models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 512
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        logits = self.classifier(x)
        probabilities = torch.softmax(logits, dim=1)
        return logits, probabilities
#END CLASS

#Create the model, we'll call it model_one.
model_one = EmotionClassifier(num_classes=7)

# Move model to device
model_one.to(device)
print(f"MultiModal model moved to {device}")

#this is just done to show a snippet of the models layout.
print(str(model_one)[:300])


MultiModal model moved to cuda:0
EmotionClassifier(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, strid


/home/data/Purdue/CSCI495/phase4/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/data/Purdue/CSCI495/phase4/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
class MultiModalEmotionClassifierB(nn.Module):
    def __init__(self, num_classes=7, vocab_size=1000, dropout_p=0.3):
        super().__init__()
        enet_out_size = 512
        
        #Image Model (Resnet18)
        self.base_image_model = torchvision.models.resnet18(pretrained=True) #Set base model
        self.features = nn.Sequential(*list(self.base_image_model.children())[:-1])

        #Text Model (Transformer) - Architecture B
        self.text_encoder = TransformerEncoder(vocab_size=vocab_size, d_model=512, nhead=8, num_layers=3)

        #Dropout Method (updated to p=0.3)
        self.dropout = nn.Dropout(p=dropout_p)
        
        # Updated classifier for 1024-D input (512 image + 512 text)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, num_classes)  # Changed from 512 to 1024
        )

    def forward(self, images, text_tokens):
        # Image Processing
        image_features = self.features(images).view(images.size(0), -1)

        # Text Processing with Transformer
        text_features = self.text_encoder(text_tokens)

        # Fusion: Concatenate [512-D image, 512-D text] → 1024-D
        fused_features = torch.cat([image_features, text_features], dim=1)

        # Dropout, randomly select p% of features to drop
        fused_features = self.dropout(fused_features)
        
        # Classify
        logits = self.classifier(fused_features)
        probabilities = torch.softmax(logits, dim=1)
        return logits, probabilities
#END CLASS

#Create the Architecture B model
# Use tokenizer's vocab size instead of hardcoded value
model_multi_b = MultiModalEmotionClassifierB(num_classes=7, vocab_size=get_vocab_size())

# Move model to device
model_multi_b.to(device)
print(f"Architecture B model moved to {device}")

#this is just done to show a snippet of the models layout.
print(str(model_multi_b)[:300])


Architecture B model moved to cuda:0
MultiModalEmotionClassifierB(
  (base_image_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(ker


In [8]:
class OurMultiModalDataSet(Dataset):
    """Multimodal dataset combining images and text for Architecture B"""
    def __init__(self, data_directory, text_dataframe, transform=None, max_text_length=15):
        self.data_image = ImageFolder(data_directory, transform=transform)
        self.text_dataframe = text_dataframe
        self.max_text_length = max_text_length
        
        # Ensure text data matches image data length
        # For now, we'll sample text randomly or use a mapping strategy
        # In production, you'd want proper image-text pairing
        
    def __len__(self):
        return len(self.data_image)
    
    def __getitem__(self, at_index):
        # Get image and label
        image, label = self.data_image[at_index]
        
        # Get corresponding text - sample from text data with matching label
        # If no exact match, use random text with same label
        matching_texts = self.text_dataframe[self.text_dataframe['label'] == label]['text']
        if len(matching_texts) > 0:
            text = matching_texts.sample(n=1).iloc[0]
        else:
            # Fallback: use any text
            text = self.text_dataframe.sample(n=1).iloc[0]['text']
        
        # Tokenize text
        text_tokens = tokenize_text(text, max_length=self.max_text_length)
        
        return image, text_tokens, label

    @property
    def classes(self):
        return self.data_image.classes
#END CLASS

# Data Loaders
Starting with images.

In [9]:
# Multimodal Data Loaders for Architecture B
batch_size = 32  # Reduced for multimodal (images + text)

# Create multimodal datasets (will be created after text data is loaded)
# For now, keep image-only loaders as backup
loader_train_images = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_valid_images = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)
loader_test_images = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

print("Image-only loaders created. Multimodal loaders will be created after text data is loaded.")

Image-only loaders created. Multimodal loaders will be created after text data is loaded.


# Text data loading

## Translation Dictionary
The text uses a different classification index versus the images, so we create a dictionary to translate between them.

In [10]:
# Text Class Number : Images Class Number
# So you can put in the text class number and get out the version of the number as the images dataset uses.
translation_dictionary = {
    0:5, #Sadness -> Sad
    1:3, #Joy -> Happy
    2:3, #Love -> Happy
    3:0, #Anger -> Angry
    4:2, #Fear -> Fear
    5:6  #Surprise -> Surprise
}

## Reading and Splitting
Actually loading

In [11]:
str_text_data_dir = "bhavikjikadara/emotions-dataset"

# Download the dataset first
dataset_path = kagglehub.dataset_download(str_text_data_dir)
print("Dataset downloaded to:", dataset_path)

# Load the CSV file from the downloaded dataset
import os
csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
if csv_files:
    csv_path = os.path.join(dataset_path, csv_files[0])
    complete_csv = pd.read_csv(csv_path)

#Read CSV into a data-frame
print("Preview of the CSV contents:")
print(complete_csv)
print("-- -- -- -- -- -- --")

#Fix class labeling missmatch
complete_csv['label'] = complete_csv['label'].replace(translation_dictionary)
print("Preview of altered CSV contents:")
print(complete_csv)
print("-- -- -- -- -- -- --")

#Split CSV into segments for Testing, Training, and Validation
from sklearn.model_selection import train_test_split

# Split: 70% train, 15% val, 15% test
train_text, temp_text, train_labels, temp_labels = train_test_split(
    complete_csv['text'], complete_csv['label'], 
    test_size=0.3, random_state=42, stratify=complete_csv['label']
)
val_text, test_text, val_labels, test_labels = train_test_split(
    temp_text, temp_labels,
    test_size=0.5, random_state=42, stratify=temp_labels
)

# Create dataframes for each split
train_text_df = pd.DataFrame({'text': train_text, 'label': train_labels})
val_text_df = pd.DataFrame({'text': val_text, 'label': val_labels})
test_text_df = pd.DataFrame({'text': test_text, 'label': test_labels})

print(f"Train: {len(train_text_df)}, Val: {len(val_text_df)}, Test: {len(test_text_df)}")


Dataset downloaded to: /home/amherscher/.cache/kagglehub/datasets/bhavikjikadara/emotions-dataset/versions/1
Preview of the CSV contents:
                                                     text  label
0           i just feel really helpless and heavy hearted      4
1       ive enjoyed being able to slouch about relax a...      0
2       i gave up my internship with the dmrg and am f...      4
3                              i dont know i feel so lost      0
4       i am a kindergarten teacher and i am thoroughl...      4
...                                                   ...    ...
416804  i feel like telling these horny devils to find...      2
416805  i began to realize that when i was feeling agi...      3
416806  i feel very curious be why previous early dawn...      5
416807  i feel that becuase of the tyranical nature of...      3
416808  i think that after i had spent some time inves...      5

[416809 rows x 2 columns]
-- -- -- -- -- -- --
Preview of altered CSV contents:
 

In [12]:
# Create Multimodal Datasets for Architecture B
dataset_mm_train = OurMultiModalDataSet(str_data_dir_train, train_text_df, transform=transform_used, max_text_length=15)
dataset_mm_valid = OurMultiModalDataSet(str_data_dir_valid, val_text_df, transform=transform_used, max_text_length=15)
dataset_mm_test = OurMultiModalDataSet(str_data_dir_test, test_text_df, transform=transform_used, max_text_length=15)

# Create multimodal data loaders
def collate_multimodal(batch):
    """Custom collate function for multimodal data"""
    images = torch.stack([item[0] for item in batch])
    text_tokens = torch.stack([item[1] for item in batch])
    labels = torch.tensor([item[2] for item in batch], dtype=torch.long)
    return images, text_tokens, labels

loader_mm_train = DataLoader(dataset_mm_train, batch_size=batch_size, shuffle=True, collate_fn=collate_multimodal)
loader_mm_valid = DataLoader(dataset_mm_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_multimodal)
loader_mm_test = DataLoader(dataset_mm_test, batch_size=batch_size, shuffle=False, collate_fn=collate_multimodal)

print(f"Multimodal datasets created:")
print(f"Train: {len(dataset_mm_train)}, Val: {len(dataset_mm_valid)}, Test: {len(dataset_mm_test)}")


Multimodal datasets created:
Train: 30023, Val: 7504, Test: 4165


# Classifier
the model itself for simple tasks of classification.

In [13]:
# Transformer Encoder for Text Processing (Architecture B)
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=8, num_layers=3, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_encoding = self._create_positional_encoding(d_model)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=2048,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Output projection to 512-D
        self.output_proj = nn.Linear(d_model, 512)
        
    def _create_positional_encoding(self, d_model, max_len=100):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)
    
    def forward(self, text_tokens):
        # Get sequence length
        seq_len = text_tokens.size(1)
        
        # Embedding + positional encoding
        embedded = self.embedding(text_tokens) * math.sqrt(self.d_model)
        embedded = embedded + self.pos_encoding[:, :seq_len, :].to(text_tokens.device)
        
        # Create attention mask for padding tokens
        attention_mask = (text_tokens != 0).float()
        
        # Transformer encoding
        transformer_output = self.transformer(embedded, src_key_padding_mask=attention_mask == 0)
        
        # Global average pooling (mean of non-padded tokens)
        mask = attention_mask.unsqueeze(-1).expand_as(transformer_output)
        masked_output = transformer_output * mask
        text_features = masked_output.sum(dim=1) / mask.sum(dim=1)
        
        # Project to 512-D
        text_features = self.output_proj(text_features)
        
        return text_features


In [14]:
class EmotionClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        # Use torchvision instead of timm
        self.base_model = torchvision.models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 512
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        output = self.classifier(x)
        return output
#END CLASS

#Create the model, we'll call it model_one.
model_one = EmotionClassifier(num_classes=7)

# Move model to device
model_one.to(device)
print(f"MultiModal model moved to {device}")

#this is just done to show a snippet of the models layout.
print(str(model_one)[:300])
        

MultiModal model moved to cuda:0
EmotionClassifier(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, strid


/home/data/Purdue/CSCI495/phase4/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/data/Purdue/CSCI495/phase4/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## MultiModal Classifier

In [15]:
# Architecture B: MultiModal Emotion Classifier with Transformer
class MultiModalEmotionClassifierB(nn.Module):
    def __init__(self, num_classes=7, vocab_size=1000, dropout_p=0.3):
        super().__init__()
        enet_out_size = 512
        
        #Image Model (Resnet18)
        self.base_image_model = torchvision.models.resnet18(pretrained=True) #Set base model
        self.features = nn.Sequential(*list(self.base_image_model.children())[:-1])

        #Text Model (Transformer) - Architecture B
        self.text_encoder = TransformerEncoder(vocab_size=vocab_size, d_model=512, nhead=8, num_layers=3)

        #Dropout Method (updated to p=0.3)
        self.dropout = nn.Dropout(p=dropout_p)
        
        # Updated classifier for 1024-D input (512 image + 512 text)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, num_classes)  # Changed from 512 to 1024
        )

    def forward(self, images, text_tokens):
        # Image Processing
        image_features = self.features(images).view(images.size(0), -1)

        # Text Processing with Transformer
        text_features = self.text_encoder(text_tokens)

        # Fusion: Concatenate [512-D image, 512-D text] → 1024-D
        fused_features = torch.cat([image_features, text_features], dim=1)

        # Dropout, randomly select p% of features to drop
        fused_features = self.dropout(fused_features)
        
        # Classify
        output = self.classifier(fused_features)
        return output
#END CLASS

#Create the Architecture B model
# Use tokenizer's vocab size instead of hardcoded value
model_multi_b = MultiModalEmotionClassifierB(num_classes=7, vocab_size=get_vocab_size())

model_multi_b.to(device)

#this is just done to show a snippet of the models layout.
print("Architecture B - Transformer-based Multimodal Model:")
print(str(model_multi_b)[:500])


Architecture B - Transformer-based Multimodal Model:
MultiModalEmotionClassifierB(
  (base_image_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=F


In [16]:
class MultiModalEmotionClassifier(nn.Module):
    def __init__(self, num_classes=7, vocab_size=1000, dropout_p=0.5):
        super().__init__()
        enet_out_size = 512
        
        #Image Model (Resnet18)
        self.base_image_model = torchvision.models.resnet18(pretrained=True) #Set base model
        self.features = nn.Sequential(*list(self.base_image_model.children())[:-1])

        #Text Model (GRU)
        self.base_text_model = nn.Embedding(vocab_size, 512, padding_idx=0)
        self.text_Encoder = nn.GRU(512,512,num_layers=1,batch_first=True,bidirectional=False)

        #Dropout Method
        self.dropout = nn.Dropout(p=dropout_p) # Just set it to the nn.method
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )

    def forward(self, images, text):
        # Image Processing
        image_features = self.features(images)

        # Text Processing
        text_embedded = self.text_embedding(text_tokens)
        text_output, hidden_layer = self.text_encoder(text_embedded)
        texture_features = text_output[:,-1,:]

        # Fusion
        fused_features = torch.cat([image_features, text_features], dim=1)

        # Dropout, randomly select p% of features to drop
        fused_features = self.dropout(fused_features)
        
        # Classify
        output = self.classifier(fused_features)
        return output
#END CLASS

#Create the model, we'll call it model_multi because its a multi modal model.
model_multi = MultiModalEmotionClassifier(num_classes=7)

model_multi.to(device)

#this is just done to show a snippet of the models layout.
print(str(model_multi)[:300])

MultiModalEmotionClassifier(
  (base_image_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kern


# Training
Queue eye of the tiger

## Architecture B Training - Multimodal Emotion Classification


In [18]:
# Training Setup for Architecture B
# Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer for multimodal model
optimizer_mm = optim.Adam(model_multi_b.parameters(), lr=0.001)

# Training parameters
number_of_epochs = 10
training_losses = []
validation_losses = []
training_accuracies = []
validation_accuracies = []

# Move model to device
model_multi_b.to(device)
print(f"Architecture B model ready for training on {device}")
print(f"Model parameters: {sum(p.numel() for p in model_multi_b.parameters()):,}")


Architecture B model ready for training on cuda:0
Model parameters: 37,043,759


In [ ]:
# Training Loop for Architecture B
print(f"Starting training for {number_of_epochs} epochs...")
print(f"Training batches: {len(loader_mm_train)}")
print(f"Validation batches: {len(loader_mm_valid)}")

total_start_time = time.time()

for epoch in range(number_of_epochs):
    epoch_start_time = time.time()
    print(f"\n=== EPOCH {epoch+1}/{number_of_epochs} ===")
    
    # Training Phase
    model_multi_b.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, text_tokens, labels in tqdm(loader_mm_train, desc=f'Epoch {epoch+1}/{number_of_epochs} - Training'):
        # Move to device
        images = images.to(device)
        text_tokens = text_tokens.to(device)
        labels = labels.to(device)
        
        # Forward pass
        optimizer_mm.zero_grad()
        outputs = model_multi_b(images, text_tokens)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer_mm.step()
        
        # Metrics
        running_loss += loss.item() * labels.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    # Training metrics
    train_loss = running_loss / len(loader_mm_train.dataset)
    train_acc = 100. * correct / total
    training_losses.append(train_loss)
    training_accuracies.append(train_acc)
    
    # Validation Phase
    model_multi_b.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, text_tokens, labels in tqdm(loader_mm_valid, desc=f'Epoch {epoch+1}/{number_of_epochs} - Validation'):
            # Move to device
            images = images.to(device)
            text_tokens = text_tokens.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model_multi_b(images, text_tokens)
            loss = criterion(outputs, labels)
            
            # Metrics
            running_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    # Validation metrics
    valid_loss = running_loss / len(loader_mm_valid.dataset)
    valid_acc = 100. * correct / total
    validation_losses.append(valid_loss)
    validation_accuracies.append(valid_acc)
    
    # Epoch summary
    epoch_time = time.time() - epoch_start_time
    total_time = time.time() - total_start_time
    
    print(f"Epoch {epoch+1}/{number_of_epochs} Summary:")
    print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%")
    print(f"  Valid - Loss: {valid_loss:.4f}, Acc: {valid_acc:.2f}%")
    print(f"  Time: {epoch_time:.2f}s | Total: {total_time:.2f}s")

# Final summary
total_training_time = time.time() - total_start_time
print(f"\n🎉 Training completed!")
print(f"Total training time: {total_training_time:.2f}s ({total_training_time/60:.1f} minutes)")
print(f"Best validation accuracy: {max(validation_accuracies):.2f}%")


# Notes

In [ ]:
# KAGGLE setup instructions:
# 1. Get Kaggle API key from https://www.kaggle.com/account
#    Go to https://www.kaggle.com/account
#    Click "Create New API Token"
#    Download kaggle.json
# 2. Place kaggle.json in ~/.kaggle/ directory
# 3. Run this notebook - datasets will download automatically


#For Model B:
    #Changed text encoder from GRU to Transformer
    #Updated dropout from p=0.5 to p=0.3
    #Updated text input to mention BPE tokenization
    #Added comment: # ADDED: BPE (Byte Pair Encoding) Tokenization for Architecture B
    #Complete Transformer encoder implementation
    #3 layers, 8 attention heads, 512-D model
    #Positional encoding and attention masking


#Edited on Friday November 8th 2025 by Andrew
#Mostly just AI generated changes to the code. Especilaly text/transformer encoder and training loop